In [223]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
import plotly.express as px
!pip install pandas_profiling
import pandas_profiling as pdp
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [224]:
df = pd.read_csv("Preference-2022-09-15.csv")

In [225]:
df

,user_id,intensity,taste_profile,brewing_method,created,updated
0,133767,medium,"[""Caramel"", ""Chocolate""]",Drip,2022-09-15 00:42:33,2022-09-15 00:42:33
1,133766,medium,"[""Chocolate"", ""Caramel""]",Espresso,2022-09-14 21:23:10,2022-09-14 21:23:38
2,133765,medium,"[""Chocolate"", ""Caramel""]",Aeropress,2022-09-14 15:20:33,2022-09-14 15:20:33
3,133763,medium,"[""Caramel"", ""Chocolate""]",Drip Bags,2022-09-14 13:51:02,2022-09-14 13:51:02
4,133761,medium,"[""None""]",Drip Bags,2022-09-14 09:29:48,2022-09-14 09:29:48
...,...,...,...,...,...,...
19884,103927,dark,"[""Chocolate""]",nespresso,2019-05-17 19:54:17,2019-05-17 19:55:43
19885,103925,medium,"[""None""]",nespresso,2019-05-17 19:08:17,2019-05-17 19:10:31
19886,103924,medium,"[""Chocolate"", ""Citrus Lemons""]",Drip,2019-05-17 18:11:50,2019-05-17 18:11:50
19887,103919,medium,"[""Chocolate"", ""Caramel"", ""Berries""]",nespresso,2019-05-17 11:52:21,2019-05-17 11:52:59


In [226]:
# For those who have 2 or more records, keep the newest record.
df.drop_duplicates(subset='user_id',keep='first',inplace=True,ignore_index=True)
df

,user_id,intensity,taste_profile,brewing_method,created,updated
0,133767,medium,"[""Caramel"", ""Chocolate""]",Drip,2022-09-15 00:42:33,2022-09-15 00:42:33
1,133766,medium,"[""Chocolate"", ""Caramel""]",Espresso,2022-09-14 21:23:10,2022-09-14 21:23:38
2,133765,medium,"[""Chocolate"", ""Caramel""]",Aeropress,2022-09-14 15:20:33,2022-09-14 15:20:33
3,133763,medium,"[""Caramel"", ""Chocolate""]",Drip Bags,2022-09-14 13:51:02,2022-09-14 13:51:02
4,133761,medium,"[""None""]",Drip Bags,2022-09-14 09:29:48,2022-09-14 09:29:48
...,...,...,...,...,...,...
19106,103927,dark,"[""Chocolate""]",nespresso,2019-05-17 19:54:17,2019-05-17 19:55:43
19107,103925,medium,"[""None""]",nespresso,2019-05-17 19:08:17,2019-05-17 19:10:31
19108,103924,medium,"[""Chocolate"", ""Citrus Lemons""]",Drip,2019-05-17 18:11:50,2019-05-17 18:11:50
19109,103919,medium,"[""Chocolate"", ""Caramel"", ""Berries""]",nespresso,2019-05-17 11:52:21,2019-05-17 11:52:59


In [227]:
pd.value_counts(df["user_id"])

133767    1
112994    1
112997    1
93717     1
113000    1
         ..
121745    1
121746    1
121748    1
113352    1
103917    1
Name: user_id, Length: 19111, dtype: int64

In [228]:
df.isnull().sum()

user_id           0
intensity         0
taste_profile     0
brewing_method    0
created           0
updated           0
dtype: int64

In [229]:
pre = list(df["taste_profile"].values)
pre

['["Caramel", "Chocolate"]',
 '["Chocolate", "Caramel"]',
 '["Chocolate", "Caramel"]',
 '["Caramel", "Chocolate"]',
 '["None"]',
 '["Citrus Lemons", "Berries", "Chocolate"]',
 '["Citrus Lemons", "Berries", "Stone Fruits"]',
 '["Caramel", "Chocolate", "Berries"]',
 '["None"]',
 '["Berries", "Chocolate", "Stone Fruits"]',
 '["Chocolate", "Caramel", "Stone Fruits"]',
 '["None"]',
 '["Chocolate"]',
 '["Caramel", "Chocolate"]',
 '["None"]',
 '["Citrus Lemons", "Caramel"]',
 '["Caramel"]',
 '["Chocolate", "Caramel"]',
 '["Chocolate", "Caramel"]',
 '["Chocolate", "Edible Flowers", "Berries"]',
 '["Chocolate", "Caramel"]',
 '["Caramel", "Chocolate", "Edible Flowers"]',
 '["None"]',
 '["Chocolate", "Berries", "Edible Flowers"]',
 '["Chocolate"]',
 '["None"]',
 '["Berries", "Caramel", "Edible Flowers"]',
 '["Chocolate", "Caramel", "Citrus Lemons"]',
 '["Edible Flowers"]',
 '["Stone Fruits", "Edible Flowers", "Berries"]',
 '["Chocolate", "Berries", "Stone Fruits"]',
 '["Chocolate", "Caramel"]',
 

In [230]:
pre_n = []

for p in pre:
    p=p[2:-2]
    pre_n.append(p.split("\", \""))



In [231]:
pre_n

[['Caramel', 'Chocolate'],
 ['Chocolate', 'Caramel'],
 ['Chocolate', 'Caramel'],
 ['Caramel', 'Chocolate'],
 ['None'],
 ['Citrus Lemons', 'Berries', 'Chocolate'],
 ['Citrus Lemons', 'Berries', 'Stone Fruits'],
 ['Caramel', 'Chocolate', 'Berries'],
 ['None'],
 ['Berries', 'Chocolate', 'Stone Fruits'],
 ['Chocolate', 'Caramel', 'Stone Fruits'],
 ['None'],
 ['Chocolate'],
 ['Caramel', 'Chocolate'],
 ['None'],
 ['Citrus Lemons', 'Caramel'],
 ['Caramel'],
 ['Chocolate', 'Caramel'],
 ['Chocolate', 'Caramel'],
 ['Chocolate', 'Edible Flowers', 'Berries'],
 ['Chocolate', 'Caramel'],
 ['Caramel', 'Chocolate', 'Edible Flowers'],
 ['None'],
 ['Chocolate', 'Berries', 'Edible Flowers'],
 ['Chocolate'],
 ['None'],
 ['Berries', 'Caramel', 'Edible Flowers'],
 ['Chocolate', 'Caramel', 'Citrus Lemons'],
 ['Edible Flowers'],
 ['Stone Fruits', 'Edible Flowers', 'Berries'],
 ['Chocolate', 'Berries', 'Stone Fruits'],
 ['Chocolate', 'Caramel'],
 ['Edible Flowers', 'Citrus Lemons', 'Berries'],
 ['Berries', 'Ch

In [232]:
l = []
s = set(l)
for p in pre_n:
    s = s|set(p)
    
s

{'',
 'Berries',
 'Caramel',
 'Chocolate',
 'Citrus Lemons',
 'Edible Flowers',
 'None',
 'Stone Fruits'}

In [233]:
m = ('Berries',
 'Caramel',
 'Chocolate',
 'Citrus Lemons',
 'Edible Flowers',
 'None',
 'Stone Fruits')

In [234]:
z = [0,0,0,0,0,0,0]
d = dict(zip(m,z))
df1 = pd.DataFrame(d, index = [0])
df1

,Berries,Caramel,Chocolate,Citrus Lemons,Edible Flowers,None,Stone Fruits
0,0,0,0,0,0,0,0


In [235]:
for p in pre_n:
    tp = z.copy()
    for i in range(7):
        if m[i] in p:
            tp[i] = 1
    tp_df = pd.DataFrame(dict(zip(m,tp)), index = [1])
    df1 = df1.append(tp_df, ignore_index = True)
    
df1

,Berries,Caramel,Chocolate,Citrus Lemons,Edible Flowers,None,Stone Fruits
0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0
2,0,1,1,0,0,0,0
3,0,1,1,0,0,0,0
4,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...
19107,0,0,1,0,0,0,0
19108,0,0,0,0,0,1,0
19109,0,0,1,1,0,0,0
19110,1,1,1,0,0,0,0


In [236]:
df2 = df1.drop(0)

In [237]:
df2

,Berries,Caramel,Chocolate,Citrus Lemons,Edible Flowers,None,Stone Fruits
1,0,1,1,0,0,0,0
2,0,1,1,0,0,0,0
3,0,1,1,0,0,0,0
4,0,1,1,0,0,0,0
5,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...
19107,0,0,1,0,0,0,0
19108,0,0,0,0,0,1,0
19109,0,0,1,1,0,0,0
19110,1,1,1,0,0,0,0


In [238]:
df2 = df2.reset_index(drop = True)
df2

,Berries,Caramel,Chocolate,Citrus Lemons,Edible Flowers,None,Stone Fruits
0,0,1,1,0,0,0,0
1,0,1,1,0,0,0,0
2,0,1,1,0,0,0,0
3,0,1,1,0,0,0,0
4,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...
19106,0,0,1,0,0,0,0
19107,0,0,0,0,0,1,0
19108,0,0,1,1,0,0,0
19109,1,1,1,0,0,0,0


“均值编码”

In [268]:
df2_to_arr = np.asarray(df2)
df2_to_list = df2_to_arr.tolist()

In [269]:
df2_to_series = pd.Series(df2_to_list)

In [270]:
df2_counts = df2_to_series.value_counts()

In [271]:
df2_counts = pd.DataFrame(df2_counts)

In [272]:
df2_counts.columns={'val'}
df2_counts.reset_index(inplace=True)

In [273]:
df2_counts.columns = {'one-hot', 'val'}
df2_counts

,one-hot,val
0,"[0, 1, 1, 0, 0, 0, 0]",2870
1,"[1, 1, 1, 0, 0, 0, 0]",2350
2,"[0, 0, 1, 0, 0, 0, 0]",1914
3,"[0, 0, 0, 0, 0, 1, 0]",1820
4,"[0, 1, 1, 0, 0, 0, 1]",1057
5,"[0, 1, 1, 0, 1, 0, 0]",987
6,"[0, 1, 0, 0, 0, 0, 0]",730
7,"[1, 0, 1, 0, 0, 0, 1]",687
8,"[1, 0, 1, 1, 0, 0, 0]",678
9,"[1, 0, 1, 0, 1, 0, 0]",594


In [275]:
insert_col = []
for i in range(len(df2_to_list)):
  for j in range(len(df2_counts)):
    if df2_to_list[i] == df2_counts['one-hot'][j]:
      insert_col.append(df2_counts['val'][j])

insert_col

[2870,
 2870,
 2870,
 2870,
 1820,
 678,
 327,
 2350,
 1820,
 687,
 1057,
 1820,
 1914,
 2870,
 1820,
 26,
 730,
 2870,
 2870,
 594,
 2870,
 987,
 1820,
 594,
 1914,
 1820,
 240,
 585,
 211,
 339,
 687,
 2870,
 285,
 2350,
 1820,
 80,
 687,
 1057,
 987,
 2350,
 1820,
 2350,
 730,
 230,
 204,
 987,
 1820,
 129,
 129,
 2870,
 211,
 285,
 2350,
 2350,
 2870,
 327,
 2870,
 1057,
 1914,
 687,
 1914,
 294,
 211,
 2350,
 285,
 987,
 294,
 339,
 121,
 1914,
 1914,
 1914,
 1820,
 2870,
 678,
 2870,
 240,
 59,
 1820,
 2350,
 1914,
 1057,
 2350,
 2870,
 135,
 2870,
 987,
 339,
 2870,
 302,
 687,
 987,
 2350,
 1820,
 240,
 585,
 220,
 285,
 678,
 585,
 220,
 2870,
 2350,
 2870,
 987,
 1820,
 129,
 327,
 585,
 1820,
 339,
 687,
 730,
 594,
 987,
 45,
 2870,
 339,
 2870,
 1820,
 2870,
 99,
 2870,
 220,
 730,
 1914,
 1820,
 730,
 135,
 339,
 2870,
 2350,
 222,
 987,
 687,
 1057,
 1820,
 987,
 1057,
 1820,
 2870,
 687,
 2350,
 1914,
 1914,
 678,
 2870,
 2870,
 204,
 585,
 2870,
 27,
 2350,
 687,
 1914

In [277]:
for i in range(len(insert_col)):
  insert_col[i] /= len(df2)

In [280]:
df2.insert(0, column='taste_profile', value=insert_col)

In [281]:
df2

,taste_profile,Berries,Caramel,Chocolate,Citrus Lemons,Edible Flowers,None,Stone Fruits
0,0.150175,0,1,1,0,0,0,0
1,0.150175,0,1,1,0,0,0,0
2,0.150175,0,1,1,0,0,0,0
3,0.150175,0,1,1,0,0,0,0
4,0.095233,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
19106,0.100152,0,0,1,0,0,0,0
19107,0.095233,0,0,0,0,0,1,0
19108,0.005913,0,0,1,1,0,0,0
19109,0.122966,1,1,1,0,0,0,0


In [286]:
df2.drop(columns=['Berries', 'Caramel', 'Chocolate', 'Citrus Lemons',	'Edible Flowers',	'None',	'Stone Fruits'],inplace=True)
df2.head()

,taste_profile
0,0.150175
1,0.150175
2,0.150175
3,0.150175
4,0.095233


In [287]:
df3 =df.iloc[:,0:4]
df3

,user_id,intensity,taste_profile,brewing_method
0,133767,medium,"[""Caramel"", ""Chocolate""]",Drip
1,133766,medium,"[""Chocolate"", ""Caramel""]",Espresso
2,133765,medium,"[""Chocolate"", ""Caramel""]",Aeropress
3,133763,medium,"[""Caramel"", ""Chocolate""]",Drip Bags
4,133761,medium,"[""None""]",Drip Bags
...,...,...,...,...
19106,103927,dark,"[""Chocolate""]",nespresso
19107,103925,medium,"[""None""]",nespresso
19108,103924,medium,"[""Chocolate"", ""Citrus Lemons""]",Drip
19109,103919,medium,"[""Chocolate"", ""Caramel"", ""Berries""]",nespresso


In [288]:
#df3 = df3.drop("taste_profile", axis = 1)

df3["intensity"][0]

for i in range(len(df3)):
    if df3["intensity"][i] == "light":
        df3["intensity"][i]=0
    elif df3["intensity"][i] == 'medium':
        df3["intensity"][i]=1
    elif df3["intensity"][i] == 'dark':
        df3["intensity"][i]=2

In [289]:
df3

,user_id,intensity,taste_profile,brewing_method
0,133767,1,"[""Caramel"", ""Chocolate""]",Drip
1,133766,1,"[""Chocolate"", ""Caramel""]",Espresso
2,133765,1,"[""Chocolate"", ""Caramel""]",Aeropress
3,133763,1,"[""Caramel"", ""Chocolate""]",Drip Bags
4,133761,1,"[""None""]",Drip Bags
...,...,...,...,...
19106,103927,2,"[""Chocolate""]",nespresso
19107,103925,1,"[""None""]",nespresso
19108,103924,1,"[""Chocolate"", ""Citrus Lemons""]",Drip
19109,103919,1,"[""Chocolate"", ""Caramel"", ""Berries""]",nespresso


In [290]:
df3 = df3.drop("taste_profile", axis = 1)
df3["intensity"]=df3["intensity"].astype('int')

In [291]:
df3

,user_id,intensity,brewing_method
0,133767,1,Drip
1,133766,1,Espresso
2,133765,1,Aeropress
3,133763,1,Drip Bags
4,133761,1,Drip Bags
...,...,...,...
19106,103927,2,nespresso
19107,103925,1,nespresso
19108,103924,1,Drip
19109,103919,1,nespresso


In [298]:
brewing_method = df3['brewing_method'].tolist()

In [299]:
brewing_series = pd.Series(brewing_method)

In [302]:
brewing_counts = brewing_series.value_counts()

In [303]:
brewing_counts

Drip            4254
nespresso       4243
Drip Bags       4183
Espresso        2307
French Press    1565
Cold Brew       1193
Stove Top        786
Aeropress        580
dtype: int64

In [305]:
brewing_counts = pd.DataFrame(brewing_counts)
brewing_counts.reset_index(inplace=True)
brewing_counts.columns = {'brewing','val'}
brewing_counts

,brewing,val
0,Drip,4254
1,nespresso,4243
2,Drip Bags,4183
3,Espresso,2307
4,French Press,1565
5,Cold Brew,1193
6,Stove Top,786
7,Aeropress,580


In [308]:
insert_col1 = []
for i in range(len(brewing_method)):
  for j in range(len(brewing_counts)):
    if brewing_method[i] == brewing_counts['brewing'][j]:
      insert_col1.append(brewing_counts['val'][j])
insert_col1

[4254,
 2307,
 580,
 4183,
 4183,
 2307,
 4183,
 4183,
 1565,
 4254,
 2307,
 4183,
 786,
 786,
 2307,
 4254,
 4183,
 2307,
 1565,
 2307,
 4254,
 4243,
 4254,
 4243,
 4243,
 2307,
 4254,
 2307,
 4254,
 2307,
 4243,
 4183,
 4243,
 4243,
 4243,
 2307,
 2307,
 1565,
 4183,
 4243,
 4254,
 786,
 4254,
 4254,
 4254,
 4183,
 2307,
 4243,
 4254,
 4183,
 4254,
 4254,
 4254,
 2307,
 580,
 1193,
 1193,
 786,
 2307,
 4183,
 4243,
 4254,
 4254,
 2307,
 4254,
 2307,
 4183,
 1565,
 2307,
 2307,
 2307,
 2307,
 1193,
 1565,
 4254,
 4243,
 4243,
 4183,
 4183,
 2307,
 2307,
 4183,
 4183,
 786,
 4183,
 2307,
 4243,
 4183,
 4183,
 2307,
 4183,
 580,
 4183,
 580,
 4254,
 786,
 2307,
 2307,
 580,
 580,
 4254,
 2307,
 4183,
 2307,
 2307,
 786,
 4254,
 4254,
 4183,
 2307,
 4254,
 2307,
 4243,
 4254,
 4243,
 4254,
 2307,
 786,
 4243,
 4183,
 2307,
 4254,
 1565,
 4243,
 4254,
 4243,
 4183,
 1193,
 4254,
 4243,
 580,
 4183,
 4183,
 2307,
 4183,
 4183,
 2307,
 4254,
 4243,
 4254,
 1193,
 2307,
 4243,
 1193,
 4183,


In [309]:
for i in range(len(insert_col1)):
  insert_col1[i] /= len(df3)

In [311]:
df3.drop(columns=['brewing_method'],inplace=True)
df3.head()

,user_id,intensity
0,133767,1
1,133766,1
2,133765,1
3,133763,1
4,133761,1


In [312]:
df3.insert(2, column='brewing_method', value=insert_col1)

In [316]:
df5 = pd.concat([df3,df2],axis = 1)
df5

,user_id,intensity,brewing_method,taste_profile
0,133767,1,0.222594,0.150175
1,133766,1,0.120716,0.150175
2,133765,1,0.030349,0.150175
3,133763,1,0.218879,0.150175
4,133761,1,0.218879,0.095233
...,...,...,...,...
19106,103927,2,0.222019,0.100152
19107,103925,1,0.222019,0.095233
19108,103924,1,0.222594,0.005913
19109,103919,1,0.222019,0.122966


In [ ]:
# df5_features = df5.iloc[:,1:]
# df5_features_arr = np.array(df5_features)

In [98]:
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score

In [101]:
# sil_score = []
# for k in range(2,8):
#   kmeans = KMeans(n_clusters=k, random_state=0).fit(df5_features_arr)
#   sil_score.append(silhouette_score(df5_features_arr,kmeans.labels_))
# plt.plot(range(2,8), sil_score, 'o-')
# plt.xlabel('k')
# plt.show()

In [102]:
# sil_score

[0.12734052865910978,
 0.1222457944525358,
 0.14208671750415025,
 0.12902952767669937,
 0.17508860482017416,
 0.17471544064346772]

In [ ]:
# pd.value_counts(df["intensity"])


medium    11248
dark       5406
light      2457
Name: intensity, dtype: int64

In [ ]:
# pd.value_counts(df["brewing_method"])

Drip            4254
nespresso       4243
Drip Bags       4183
Espresso        2307
French Press    1565
Cold Brew       1193
Stove Top        786
Aeropress        580
Name: brewing_method, dtype: int64

In [ ]:
# pd.value_counts(df["brewing_method"]).index

Index(['Drip', 'nespresso', 'Drip Bags', 'Espresso', 'French Press',
       'Cold Brew', 'Stove Top', 'Aeropress'],
      dtype='object')